In [1]:
import sail.core
import json
import getpass

# password = getpass.getpass(prompt="Password: ")
password = "SailPassword@123"

sail.core.login("lbart@igr.com", password, 6200, "40.71.25.110")

sail.core.load_safe_objects("./SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"

job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

#second_job_id = sail.core.run_job(dg_test_object_guid)
#print(second_job_id)

datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
else:
    print("No loaded datasets")
    
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
else:
    print("No digital contracts")

{48A9BCF6-6F9B-4A3F-8E2B-AA52B093BEE9}
{219AA7E2-6253-5549-BBA8-89DF5D77639D}
{15FA536F-B8A6-4327-A9CF-E63C5C391F87}


In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

In [2]:
first_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")
#second_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")

print(first_provision)
#print(second_provision)

{"SCNGuid":"{4328C9E3-B992-4C06-997E-C3ABAD33DC60}","Status":true}


In [4]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(1000))

{"AllDone":false,"Failed":{},"InProgress":{"{4328C9E3-B992-4C06-997E-C3ABAD33DC60}":""},"Succeeded":{}}


In [ ]:
my_table_guid = "2534C8FA-0E1A-D442-95A6-8ECB11A39746"
set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)

print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))

In [ ]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")

pull_data_result = sail.core.pull_data(data_id)
print(pull_data_result)

In [ ]:
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
data_frame = "";
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {}".format(un_serialized_value))
            data_frame = un_serialized_value
            
    message_response = sail.core.wait_for_data(500)

In [ ]:
safe_functions = sail.core.get_safe_functions()
if ( safe_functions is not None):
    json_safe_functions = json.loads(safe_functions)
    print(json.dumps(json_safe_functions,indent=4))
else:
    print("No safe functions")

In [ ]:
sail.core.exit_current_session()

In [ ]:
sail.core.deprovision_digital_contract(provision_dc)

In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
tables = sail.core.get_tables()
if ( tables is not None):
    json_tables = json.loads(tables)
    print(json.dumps(json_tables, indent=4))
else:
    print("No tables")


In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [5]:
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
my_table_guid = "2534C8FA-0E1A-D442-95A6-8ECB11A39746"

increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)
print(increment_job_id)

pull_data_id = "{}.{}".format(increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

pull_data_result = sail.core.pull_data(pull_data_id)
print(pull_data_result)

my_int_guid = sail.core.push_user_data(851)
set_response = sail.core.set_parameter(increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", my_int_guid)
set_response = sail.core.set_parameter(increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)
print(my_int_guid)

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

{48CD466F-5AF9-4616-9A6C-FD6D5F6A0D90}
Success
{57D0645A-C3DD-4150-8F48-AD00E9B0109A}
Job 48CD466F5AF946169A6CFD6D5F6A0D90 is running
Job 48CD466F5AF946169A6CFD6D5F6A0D90 is done
Data received 48CD466F5AF946169A6CFD6D5F6A0D90.51DD83834C2A41D6BB1F49889EA0A0AC 458


In [8]:
second_increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)

# We want the second job's value to be the result of the first job
second_pull_data_id = "{}.{}".format(second_increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

second_pull_data_result = sail.core.pull_data(second_pull_data_id)

set_response = sail.core.set_parameter(second_increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", pull_data_id)
set_response = sail.core.set_parameter(second_increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

Job 4AC52601DCEB4921AF547BAAC9F310BD is running
Job 4AC52601DCEB4921AF547BAAC9F310BD is done
Data received 4AC52601DCEB4921AF547BAAC9F310BD.51DD83834C2A41D6BB1F49889EA0A0AC 459


In [7]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(500)

In [ ]:
pull_data_id = "{}.{}".format(job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

pull_data_result = sail.core.pull_data(pull_data_id)
print(pull_data_result)

In [ ]:
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
    print(json.dumps(json_digital_contracts, indent=4))
else:
    print("No digital contracts")

In [ ]:
datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
    print(json.dumps(json_datasets, indent=4))
else:
    print("No loaded datasets")